In [1]:
import datetime
import requests
import json

In [2]:
def callApi(date):
    key = "2b423d0c90e52bb805ccf6af1f305d7c"
    latitude = "13.8282"
    longitude = "100.614"
    time = str(date).replace(" ", "T")
    url = "https://api.darksky.net/forecast/{}/{},{},{}".format(key, latitude, longitude, time)
    
    response = requests.get(url=url)
    return response

In [3]:
def one_month(month, year):
    date = datetime.datetime(year=year, month=month, day=29)
    nextMonth = date.month + 1
    while(date.month != nextMonth):
        yield date
        date = date + datetime.timedelta(days=1)   
        
def one_day(date):
    tomorrow = date + datetime.timedelta(days=1)
    while(date.day != tomorrow.day):
        yield date
        date = date + datetime.timedelta(minutes=15)   

In [4]:
def write_json(filename, data):
    with open("tongsdata/" + filename + ".json", "w", encoding='utf-8') as fout:
        fout.write(json.dumps(data))

___

In [9]:
#  LOAD INTO FILES

for day in one_month(11, 2018):
    day_result = {}
    filename = day.strftime("%d-%m-%Y")
    for minute in one_day(day):
        result = callApi(minute)
        day_result.update({ minute.strftime("%H:%M"): result.json()})  #  key is time of the day
    write_json(filename=filename, data=day_result)  #  filename is date    

KeyboardInterrupt: 

___

In [1]:
import os
import pandas as pd
import json
import datetime

In [2]:
def createDayDataframe(filename):
    path = "tongsdata/"
    filedate = filename.split(".")[0].split("-")

    with open(path + filename, 'r', encoding='utf-8') as fin:
        data_json = json.loads(fin.read())

    #  Create X DataFrame
    todayData = {time: data_json[time]['currently'] for time in data_json}
    todayData_DF = pd.DataFrame(todayData).T

    #  Create y DataFrame
    energyBalanceFilename = "energyBalance/Energy_Balance_{}_{}_{}.csv".format(filedate[2], filedate[1], filedate[0])
    energyBalance = pd.read_csv(path + energyBalanceFilename, delimiter=";")

    energyBalance_DF = pd.DataFrame(index=energyBalance.iloc[:-1,0].apply(lambda x: x[2:-1]))
    energyBalance_DF['PV Power Generation'] = energyBalance['PV power generation / Mean values [W]  '].values[:-1]
    energyBalance_DF = energyBalance_DF[energyBalance_DF != ' ']
    energyBalance_DF = energyBalance_DF.dropna()
    
    #  Join X y DataFrame
    data = todayData_DF.join(energyBalance_DF)
    return data

def cleanDataframe(masterDF):
    masterData_cleaned = masterDF.reset_index()
    masterData_cleaned = masterData_cleaned.rename(columns={'index': 'time_of_day'})
    masterData_cleaned = masterData_cleaned[masterData_cleaned['PV Power Generation'].notna()]
    masterData_cleaned['month'] = masterData_cleaned['time'].apply(datetime.datetime.fromtimestamp).apply(lambda x: x.month)
    masterData_cleaned['cloudCover'] = masterData_cleaned['cloudCover'].fillna(value=masterData_cleaned['cloudCover'].mean())
    masterData_cleaned['PV Power Generation'] = masterData_cleaned['PV Power Generation'].apply(lambda x: x.replace(",", "")).astype('float')
    return masterData_cleaned

In [3]:
def featureEngineering(masterDF):
    masterDF = masterDF.drop(columns=['icon', 'summary', 'time', 'windBearing', 'windGust'])
    masterDF = pd.get_dummies(masterDF, columns=['time_of_day', 'precipType', 'uvIndex', 'month'])
    return masterDF

In [4]:
files = os.listdir("tongsdata/")

#files.remove(".DS_Store")
files.remove("energyBalance")

masterData = pd.DataFrame()

for file in files:
    r = createDayDataframe(file).loc['06:30':'18:00']
    print(file, r.shape)
    masterData = pd.concat([masterData, r], axis=0)
    
masterData = cleanDataframe(masterData)
masterData = featureEngineering(masterData)

01-11-2018.json (47, 15)
01-12-2018.json (47, 15)
02-11-2018.json (47, 13)


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


02-12-2018.json (47, 15)
03-11-2018.json (47, 15)
03-12-2018.json (47, 14)
04-11-2018.json (47, 14)
04-12-2018.json (47, 14)
05-11-2018.json (47, 14)
05-12-2018.json (47, 14)
06-11-2018.json (47, 15)
06-12-2018.json (47, 15)
07-11-2018.json (47, 14)
07-12-2018.json (47, 14)
08-11-2018.json (47, 15)
08-12-2018.json (47, 15)
09-11-2018.json (47, 14)
09-12-2018.json (47, 15)
10-11-2018.json (47, 15)
10-12-2018.json (47, 14)
11-11-2018.json (47, 14)
12-11-2018.json (47, 15)
13-11-2018.json (47, 14)
14-11-2018.json (47, 15)
15-11-2018.json (47, 15)
16-11-2018.json (47, 15)
17-11-2018.json (47, 14)
18-11-2018.json (47, 15)
19-11-2018.json (47, 15)
20-11-2018.json (47, 14)
21-11-2018.json (47, 15)
22-11-2018.json (47, 15)
23-11-2018.json (47, 15)
24-11-2018.json (47, 19)
25-11-2018.json (47, 19)
26-11-2018.json (47, 19)
27-11-2018.json (47, 19)
28-11-2018.json (47, 19)
29-11-2018.json (47, 15)
30-11-2018.json (47, 14)


y
    'PV Power Generation'

X
    'time_of_day'         ->  one-hot
    'apparentTemperature' ->  OK
    'cloudCover'          ->  OK 
    'dewPoint'            ->  OK
    'humidity'            ->  OK
    'icon'                ->  drop
    'precipType'          ->  one-hot
    'summary'             ->  drop
    'temperature'         ->  OK
    'time'                ->  drop
    'uvIndex'             ->  one-hot
    'visibility'          ->  OK
    'windBearing'         ->  drop (NN NE EE ...)
    'windGust'            ->  drop (what is wind gust?)
    'windSpeed'           ->  OK
    'month'               ->  one-hot (Should do season or quarter?)

___

In [ ]:
masterData.info()

In [12]:
features = list(masterData.drop(columns=['PV Power Generation']).columns)

In [13]:
masterData = masterData.dropna()

X = masterData.drop(columns=['PV Power Generation']).astype('float').values
y = masterData['PV Power Generation'].astype('float').values


In [14]:
masterData.to_csv("masterData1.csv")

In [15]:
X.shape

(250, 70)

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

ImportError: No module named 'sklearn.__check_build._check_build'
___________________________________________________________________________
Contents of C:\Users\User\Anaconda3\lib\site-packages\sklearn\__check_build:
setup.py                  _check_build.cp37-win_amd64.pyd__init__.py
__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [298]:
from sklearn.linear_model import LinearRegression

LR = LinearRegression()
LR = LR.fit(X_train, y_train)

y_hat = LR.predict(X_test)

In [299]:
from sklearn.metrics import mean_squared_error, r2_score
import math

math.sqrt(mean_squared_error(y_test, y_hat))

382.25264966030744

In [300]:
result = pd.DataFrame()
result['Actual'] = y_test
result['Prediction'] = y_hat

In [301]:
result
r.to_csv("masterData1.csv")

,Actual,Prediction
0,16.0,-40.156250
1,1480.0,1740.796875
2,103.0,336.000000
3,233.0,510.578125
4,1381.0,1447.375000
5,399.0,565.843750
6,2093.0,2139.906250
7,54.0,-186.093750
8,2400.0,1289.843750
9,1506.0,1822.515625


In [302]:
from tpot import TPOTRegressor

tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: -90309.5535077827
Generation 2 - Current best internal CV score: -90309.5535077827
Generation 3 - Current best internal CV score: -90309.5535077827
Generation 4 - Current best internal CV score: -90309.5535077827
Generation 5 - Current best internal CV score: -90309.5535077827

Best pipeline: GradientBoostingRegressor(CombineDFs(input_matrix, input_matrix), alpha=0.9, learning_rate=0.1, loss=lad, max_depth=9, max_features=1.0, min_samples_leaf=2, min_samples_split=8, n_estimators=100, subsample=0.9000000000000001)
-76798.2979484793


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [383]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

training_features, testing_features, training_target, testing_target = train_test_split(X, y)

# Average CV score on the training set was:-90309.5535077827
exported_pipeline = make_pipeline(
    GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss="lad", max_depth=9, max_features=1.0, min_samples_leaf=2, min_samples_split=8, n_estimators=100, subsample=0.9000000000000001)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
mean_squared_error(results, testing_target)

85858.17397835421

In [384]:
featuresImportance = pd.DataFrame()
featuresImportance['Feature Name'] = features
featuresImportance['Importance'] = exported_pipeline.steps[0][1].feature_importances_

featuresImportance.sort_values('Importance', ascending=False).head()

,Feature Name,Importance
6,windSpeed,0.140469
3,humidity,0.126531
2,dewPoint,0.120318
0,apparentTemperature,0.098046
4,temperature,0.090284


In [360]:
result = pd.DataFrame()
result['Actual'] = testing_target
result['Prediction'] = results

# result

___

In [394]:
edaData = masterData

In [397]:
edaData['windSpeed']

0      3.21
1      3.21
2      3.25
3      3.25
4      3.24
5      3.25
6      3.25
7      2.93
8      3.28
9      4.13
10     5.25
11     5.44
12     5.62
13     5.81
14        6
15     6.73
16     7.46
17     8.19
18     8.92
19     8.75
20     8.65
21     8.61
22     8.66
23      8.7
24     8.78
25     8.89
26     9.04
27     8.78
28     8.52
29     8.27
       ... 
439    6.56
440    6.63
441    6.73
442    6.29
443    5.88
444    5.49
445    5.13
446    5.41
447     5.7
448       6
449     6.3
450    6.89
451    7.56
452    8.27
453    9.03
454    8.41
455    7.81
456    7.26
457    6.75
458    6.99
459    7.34
460    7.77
461    8.27
462    7.85
463    7.45
464    7.08
465    6.75
466    6.73
467    6.72
468    6.73
Name: windSpeed, Length: 465, dtype: object